# Equation de la chaleur

On considère l'équation de la chaleur dans le cas 1D :


\begin{equation}
\left\{
\begin{aligned}
& \partial_t u(x,t) - D \, \partial_{xx} u(x,t) = 0 \quad \text{dans} \; R\\
& u(x,0)  =  \delta_0(x),
\end{aligned}
\right.
\end{equation}

où $\delta_0(x)$ désigne la fonction delta de Dirac à l'origine en $x=0$.

Avec des conditions aux limites de type Neumann homogène , la solution analytique de l'équation précedente s'écrit :

\begin{equation}
u(x,t) = \frac{1}{\sqrt{\pi D t}} \exp(-\frac{x^2}{\sqrt{4Dt}})
\end{equation}

In [ ]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import PrintfTickFormatter

from mylib.heat_model import heat_model

import mylib.integration as integration

output_notebook(hide_banner=True)

## Valeurs propres du Laplacien

In [ ]:
def show_eigen_values(): 
    
    xmin = -5.
    xmax = 5.
    # nb of points including boundary conditions
    nxib = 2001
    nx = nxib-2
    dx = (xmax-xmin)/(nxib-1)
    
    discretized_eig = np.empty(nx)
    laplacian_eig = np.empty(nx)
    for i in range(nx):
        discretized_eig[i] = -(4/(dx*dx)) * np.sin((i * np.pi)/(2*(nx+1))) * np.sin((i * np.pi )/(2*(nx+1)))
        laplacian_eig[i] = -((np.pi*np.pi)*(i*i))/(4*((xmax-xmin)/2)*((xmax-xmin)/2))

    fig_eig = figure(x_range=(0, nx+1), plot_height=450, plot_width=950)
    fig_eig.x(np.linspace(0,nx-1,nx), discretized_eig, legend="eigen values of the discretized Laplacian")
    fig_eig.line(np.linspace(0,nx-1,nx), laplacian_eig, color="red", line_width=2, legend="eigen values of the Laplacian")
    fig_eig.yaxis[0].formatter = PrintfTickFormatter(format="%f")

    show(fig_eig)
    
show_eigen_values()

## Intégration avec Rock4 

In [ ]:
def plot_rock_sol():
    
    xmin = -5.
    xmax = 5.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.001
    tend = 0.201
    d = 1.    
            
    hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_rock  = hm.fcn_rock
    fcn_radau  = hm.fcn_radau
    fcn_exact  = hm.fcn_exact
                        
    # initial solution    
    yini = hm.fcn_exact(tini)
    
    # rock4 solution
    tol = 1.e-8
    sol = integration.rock4(tini, tend, yini, fcn_rock, rtol=tol, atol=tol)  
    
    # quasi-exact solution
    sol_qexa = integration.radau5(tini, tend, yini, fcn_radau, njac=1, rtol=1.e-12, atol=1.e-12)
    yerr_qexa = np.abs(sol_qexa.y - sol.y)
    
    # exact solution
    yexa = fcn_exact(tend)
    #for i, yi in enumerate(yexa):
    #    print(i, yi)
    yerr_exa = np.abs(yexa - sol.y)
    
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)
    
    fig_sol = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution")
    fig_sol.x(x, sol.y)
    
    fig_err = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Global error")
    fig_err.x(x, yerr_exa, legend="Fundamental solution as exact solution", color="green")  
    fig_err.x(x, yerr_qexa,legend="Quasi exact solution (Radau5) as exact solution", color="crimson")

    show(column(fig_sol, fig_err))
        
    print(f"Number of function evaluations : {sol.nfev:d}")
    print(f"Number of computed steps : {sol.nstep:d}")
    print(f"Number of accepted steps : {sol.naccpt:d}")
    print(f"Number of rejected steps : {sol.nrejct:d}")
    print(f"Maximum number of stage used : {sol.nstage:d}")
    
plot_rock_sol()

## Intégration avec Radau5

In [ ]:
def plot_radau_sol():

    xmin = -5.
    xmax = 5.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.001
    tend = 0.201
    d = 1.    
            
    hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_rock  = hm.fcn_rock
    fcn_radau  = hm.fcn_radau
    fcn_exact  = hm.fcn_exact
                        
    # initial solution    
    yini = hm.fcn_exact(tini)
    
    # radau5 solution
    tol = 1.e-6
    sol = integration.radau5(tini, tend, yini, fcn_radau, njac=1, rtol=tol, atol=tol)
    
    # quasi-exact solution
    sol_qexa = integration.radau5(tini, tend, yini, fcn_radau, njac=1, rtol=1.e-12, atol=1.e-12)
    yerr_qexa = np.abs(sol_qexa.y - sol.y)
    
    # exact solution
    yexa = fcn_exact(tend)
    yerr_exa = np.abs(yexa - sol.y)
    
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)

    fig_sol = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution")
    fig_sol.x(x, sol.y)
    fig_err = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Global error")
    fig_err.x(x, yerr_exa, legend="Fundamental solution as exact solution", color="green")  
    fig_err.x(x, yerr_qexa,legend="Quasi exact solution (Radau5) as exact solution", color="crimson")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err))
    
    print(f"Number of function evaluations : {sol.nfev:d}")
    print(f"Number of jacobian evaluations : {sol.njev:d}")
    print(f"Number of computed steps : {sol.nstep:d}")
    print(f"Number of accepted steps : {sol.naccpt:d}")
    print(f"Number of rejected steps : {sol.nrejct:d}")
    print(f"Number of LU decompositions : {sol.ndec:d}")
    print(f"Number of forward-backward substitutions : {sol.nsol:d}")

    
plot_radau_sol()